In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from tqdm import tqdm
import sys
sys.path.append('/Users/vishwajit/.pyenv/versions/3.7.3/lib/python3.7/site-packages/')
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import scipy
from scipy.signal import find_peaks

In [133]:
location = pd.read_csv('location.csv')
acc = pd.read_csv('samples.csv')
run = pd.read_csv('run.csv')
maneuvers = pd.read_csv('maneuvers.csv')
maneuvers = maneuvers.set_index(['run_id','turn_id'],drop=True)

In [134]:
# timestamp has accuracy till milliseconds. Hence divide by 1000 and convert it to datetime object
acc['time'] = [datetime.fromtimestamp(x/1000) for x in acc['timestamp']]
location['time'] = [datetime.fromtimestamp(x/1000) for x in location['timestamp']]

In [135]:
# Feature Extraction based on the paper on measuring driver behavior with smartphones
for run_id in tqdm(range(1,10)):
    turn_ids = list(maneuvers.loc[run_id,:].index) # find the list of turn ids corresponding to given a run_id
    for turn in turn_ids:
        # separate the x axis accelerometer data corresponding to the turn
        acc_x = np.array(acc[(acc['turn_id']==turn)&(acc['provider']=='acc')&(acc['run_id']==run_id)]['x_axis'])
        time = list(acc[(acc['turn_id']==turn)&(acc['provider']=='acc')&(acc['run_id']==run_id)]['time'])
        
        t = []
        for i in range(len(acc_x)):
            t.append((time[i]-time[0]).total_seconds()) # find the time difference between starting of the event and given sample
        coefs = np.polyfit(t,acc_x,5) # fit a fifth order polynomial to the acceleration data 
        for i in range(6):
            maneuvers.loc[(run_id,turn),'acc_'+str(i)] = coefs[i] # use the 6 coefficients as 6 features
        
        #minimum peak to maximum peak amplitude
        peaks,_ = find_peaks(acc_x)
        peak_acc_x = acc_x[peaks]
        try:
            maneuvers.loc[(run_id,turn),'min_to_max_peak_acc'] = max(peak_acc_x) - min(peak_acc_x) 
        except:
            maneuvers.loc[(run_id,turn),'min_to_max_peak_acc'] = 0
        maneuvers.loc[(run_id,turn),'min_acc_x'] = min(acc_x) #minimum amplitude 
        maneuvers.loc[(run_id,turn),'max_acc_x'] = max(acc_x) #maximum amplitude
        maneuvers.loc[(run_id,turn),'mean_acc_x'] = np.mean(acc_x) #average amplitude
        T = t[-1]
        N = len(t)
        maneuvers.loc[(run_id,turn),'sig_energy_acc'] = T/N * np.sum(acc_x**2) #Signal Energy
        
        # Repeat all the features for gyroscope data
        ang_z = np.array(acc[(acc['turn_id']==turn)&(acc['provider']=='gyr')&(acc['run_id']==run_id)]['z_axis'])
        time = list(acc[(acc['turn_id']==turn)&(acc['provider']=='gyr')&(acc['run_id']==run_id)]['time'])
        
        t = []
        for i in range(len(time)):
            t.append((time[i]-time[0]).total_seconds())
        coefs = np.polyfit(t,ang_z,5)
        for i in range(6):
            maneuvers.loc[(run_id,turn),'ang_'+str(i)] = coefs[i]
        
        maneuvers.loc[(run_id,turn),'min_ang_z'] = min(ang_z)
        maneuvers.loc[(run_id,turn),'max_ang_z'] = max(ang_z)
        maneuvers.loc[(run_id,turn),'mean_ang_z'] = np.mean(ang_z)
        
        peaks,_ = find_peaks(ang_z)
        peak_ang_z = ang_z[peaks]
        try:
            maneuvers.loc[(run_id,turn),'min_to_max_peak_ang'] = max(peak_ang_z) - min(peak_ang_z)
        except:
            maneuvers.loc[(run_id,turn),'min_to_max_peak_ang'] = 0
            
        T = t[-1]
        N = len(t)
        maneuvers.loc[(run_id,turn),'sig_energy_ang'] = T/N * np.sum(ang_z**2)
        maneuvers.loc[(run_id,turn),'integral_wz'] = scipy.integrate.simps(ang_z,dx=0.2E-06) #Integral of gyroscope signal

100%|██████████| 9/9 [01:22<00:00,  9.20s/it]


In [136]:
# XGBoost model to classify the severity of the bend
maneuvers['direction_'] = [1 if x=='left' else 0 for x in maneuvers['direction']]
X = maneuvers[['velocity','max_ang_z','mean_ang_z','direction_',
               'min_to_max_peak_ang','min_ang_z','sig_energy_ang','integral_wz']]
y = maneuvers['severity']
Skf = StratifiedKFold(n_splits=3)
valid_scores = []

for train_idx, val_idx in Skf.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[val_idx]
    
    severity_model = XGBClassifier(
        n_estimators=500,
        max_depth=3,
        learning_rate=0.01,
        objective='binary:logistic',
        eval_metric='auc'
    )
    severity_model.fit(X_train, y_train)
    
    test_preds = severity_model.predict(X_test)
    valid_scores.append(accuracy_score(y_test, test_preds))
    
print(valid_scores)

[0.8992248062015504, 0.8992248062015504, 0.9767441860465116]


In [147]:
maneuvers[['severity_1','severity_2','severity_3']] = pd.get_dummies(maneuvers.severity,prefix='severity')

In [157]:
# XGBoost model to classify aggressive driving
X = maneuvers[['velocity','max_acc_x','mean_acc_x','direction_','acc_0','acc_1','acc_2','acc_3','acc_4','acc_5',
               'min_to_max_peak_acc','min_acc_x','severity_1','severity_2','severity_3','sig_energy_acc','integral_wz',
               'max_ang_z','mean_ang_z','min_to_max_peak_ang','min_ang_z','sig_energy_ang']]
y = maneuvers['aggressive']
Skf = StratifiedKFold(n_splits=3)
valid_scores = []

for train_idx, val_idx in Skf.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[val_idx]

    severity_model = XGBClassifier(
        n_estimators=1000,
        max_depth=6,
        learning_rate=0.01,
        objective='binary:logistic',
        eval_metric='auc'
    )
    severity_model.fit(X_train, y_train)

    test_preds = severity_model.predict(X_test)
    valid_scores.append(accuracy_score(y_test, test_preds))

print(valid_scores)

[0.8604651162790697, 0.7751937984496124, 0.7131782945736435]
